## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-27-16-50-58 +0000,startribune,Live: Two children dead in shooting at Minneap...,https://www.startribune.com/minneapolis-shooti...
1,2025-08-27-16-50-00 +0000,wsj,Two Children Killed in Minneapolis Catholic Sc...,https://www.wsj.com/us-news/minnesota-school-s...
2,2025-08-27-16-48-00 +0000,wsj,Sompo Holdings to Buy Aspen Insurance for $3.5...,https://www.wsj.com/articles/sompo-holdings-to...
3,2025-08-27-16-47-49 +0000,wapo,"Live updates: Two children dead, 17 injured in...",https://www.washingtonpost.com/nation/2025/08/...
4,2025-08-27-16-46-11 +0000,bbc,Ryanair to increase oversized bag bonus for staff,https://www.bbc.com/news/articles/c99mdv3y431o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
60,trump,71
37,new,19
135,gaza,15
178,president,14
707,cook,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
245,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...,181
263,2025-08-26-22-42-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,168
264,2025-08-26-22-33-58 +0000,nypost,Fed governor Lisa Cook hires Hunter Biden lawy...,https://nypost.com/2025/08/26/us-news/fed-gove...,152
307,2025-08-26-20-04-00 +0000,wsj,President Trump has told advisers that he want...,https://www.wsj.com/politics/policy/trump-weig...,145
302,2025-08-26-20-16-50 +0000,nypost,Fed breaks silence on Trump’s firing of Lisa C...,https://nypost.com/2025/08/26/business/fed-bre...,141


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
245,181,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...
3,67,2025-08-27-16-47-49 +0000,wapo,"Live updates: Two children dead, 17 injured in...",https://www.washingtonpost.com/nation/2025/08/...
275,48,2025-08-26-21-51-00 +0000,wsj,Canada’s manufacturers and wholesalers look to...,https://www.wsj.com/articles/canada-factory-wh...
233,48,2025-08-27-01-00-00 +0000,wsj,The first Black woman to serve as a Federal Re...,https://www.wsj.com/economy/central-banking/li...
193,47,2025-08-27-07-04-00 +0000,wsj,Israeli Troops Targeted a Camera in Gaza Hospi...,https://www.wsj.com/world/middle-east/israeli-...
151,45,2025-08-27-10-00-00 +0000,nypost,Grassley insists to ‘Pod Force One’ that Biden...,https://nypost.com/2025/08/27/us-news/grassley...
82,43,2025-08-27-14-10-02 +0000,nypost,Three people suspected of carrying out covert ...,https://nypost.com/2025/08/27/world-news/denma...
87,42,2025-08-27-13-51-00 +0000,nypost,Trump administration plans to reclaim control ...,https://nypost.com/2025/08/27/us-news/trump-ad...
256,37,2025-08-26-23-05-09 +0000,nypost,"Cracker Barrel reverses course on new logo, br...",https://nypost.com/2025/08/26/business/cracker...
114,35,2025-08-27-12-11-15 +0000,wapo,"Man gets 89 days in prison, for every day he f...",https://www.washingtonpost.com/nation/2025/08/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
